## Extending the capabilities of our model

An LLM is a very capable tool, but only to the extent of the knowledge or information it has been trained on. After all, you only know what you know, right? But what if you need to ask a question that is not in the training data? Or what if you need to ask a question that is not in the training data, but is related to it?

There are different ways to solve this problem, depending on the resources you have and the time or money you can spend on it. Here are a few options:

- Fully retrain the model to include the information you need. For an LLM, it's only possible for a handful of companies in the world that can afford literally thousands of GPUs running for weeks.
- Fine-tune the model with this new information. This requires way less resources, and can usually be done in a few hours or minutes (depending on the size of the model). However as it does not fully retrain the model, the new information may not be completely integrated in the answers. Fine-tuning excels at giving a better understanding of a specific context or vocabulary, a little bit less on injecting new knowledge. Plus you have to retrain and redeploy the model anyway any time you want to add more information.
- Put this new information in a database and have the parts relevant to the query retrieved and added to this query as a context before sending it to the LLM. This technique is called **Retrieval Augmented Generation, or RAG**. It is interesting as you don't have to retrain or fine-tune the model to benefit of this new knowledge, that you can easily update at any time.

We have already prepared a Vector Database using [Amazon Bedrock Knowledge Bases](https://aws.amazon.com/bedrock/knowledge-bases/), where we have stored the content of the [California Driver's Handbook](https://www.dmv.ca.gov/portal/handbook/california-driver-handbook/).

In this Notebook, we are going to use RAG to **make some queries about a Claim** and see how this new knowledge can help without having to modify our LLM.

### Library imports

First we will import the libraries we need, they are already installed on our workbench image so no need to run `pip install`.

In [ ]:
import json
import warnings
from langchain_aws import ChatBedrockConverse
from langchain_aws.retrievers import AmazonKnowledgeBasesRetriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

warnings.filterwarnings("ignore")

### Langchain elements

Again, we are going to use Langchain to define our task pipeline.

First, the **LLM** where we will send our queries.

In [ ]:
llm = ChatBedrockConverse(
    model_id="us.amazon.nova-pro-v1:0",
    max_tokens=512,
    temperature=0.01,
    top_p=0.92,
)

Then the connection to the **Amazon Bedrock Knowledge Base** where we have prepared and stored the California Driver Handbook.

In [ ]:
retriever = AmazonKnowledgeBasesRetriever(
    knowledge_base_id="FXUBIKDGVG",
    retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 5}},
)

We will now define the **template** to use to make our query. Note that this template now contains a **References** section. That's where the documents returned from the knowledge base will be injected.

In [8]:
template="""<s>[INST]
You are a helpful, respectful and honest assistant named "Parasol Assistant".
You will be given a claim summary, references to provide you with information, and a question.
You must answer the question based as much as possible on this claim with the help of the references.
Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>

Claim Summary:
{claim}

References:
{{context}}

Question: {{question}} [/INST]
"""

We are now ready to query the model!

In the `claims` folder we have JSON files with examples of claims that could be received. We are going to read the first claim and ask a question related to it.

In [ ]:
# Read the claim and put its content in the "claim" variable

filename = 'claims/claim1.json'

# Opening JSON file
with open(filename, 'r') as file:
    data = json.load(file)
claim = data["content"]

### First test, no additional knowledge

Let's start with a first query about the claim, but without help from our knowledge base.

In [ ]:
# Create and send our query.

query = "Was Daniel allowed to pass at the red light?"

# Quick hack to reuse the same template with a different type of query.
prompt_template = template.format(claim=claim).format(context="", question=query)
prompt = PromptTemplate.from_template(prompt_template)
conversation = prompt | llm
resp = conversation.invoke({"input":""})

print(resp.content)

We can see that the answer is valid. Here the model is using its general understanding of traffic regulation.

### Second test, with added knowledge

We will use the same prompt and query, but this time the model will have access to some references from the California's Driver Handbook.

In [ ]:
# Create and send our query.
query = "Was Daniel allowed to pass at the red light?"

prompt_template = template.format(claim=claim)
prompt = PromptTemplate.from_template(prompt_template)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

resp = rag_chain.invoke(query)
print(resp.content)

That is pretty neat! Now the model refers directly to a source stating that **a red traffic signal light means "STOP."**.

That's it! We now know how to complement our LLM with some external knowledge!